Best Model i.e LSTM on Hist. of pixels (16 bin) is used to make predictions.

Success and Failure Cases are shown.

Visualization of Training and Test data both ground truth and predictions on google maps is also shown.

Please replace plot.api_key with your own key.

In [1]:
import math
from pandas import DataFrame
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from numpy import array

from keras.layers import Convolution2D, MaxPooling2D, Flatten, Reshape,Conv2D
from keras.models import Sequential
from keras.layers.wrappers import TimeDistributed
from keras.utils import np_utils

import numpy as np
import cv2
from keras.preprocessing.image import img_to_array


import matplotlib.pyplot as plt
%matplotlib inline

import pandas
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten,LSTM
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D

from keras.preprocessing.image import ImageDataGenerator
from keras import models
from numpy import array


from keras import backend as K
from sklearn.metrics import mean_absolute_error
from keras import optimizers
from keras.layers import Bidirectional


/Users/SindhuPhani/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
results_rmse = []
results_mae = []
results_std = []
import numpy

# Model 1 : LSTM with 16 bins hist of pixel vals

num_bins=16

# Model 1 : LSTM with 16 bins hist of pixel vals

In [17]:
model = Sequential() 
trainX=[]
trainY=[]
testX=[]
testY=[]

numpy.random.seed(3)
time_steps=19
# load the dataset
dataframe = pandas.read_csv('./Trainold.csv')
dataset = dataframe.values

scaler = MinMaxScaler(feature_range=(0, 1))


# we group by day so we can process a video at a time.
grouped = dataframe.groupby(dataframe.VidName)

per_vid = []
for _, group in grouped:
    per_vid.append(group)


print(len(per_vid))


# generate sequences a vid at a time
for i,vid in enumerate(per_vid):
    histValuesList=[]
    scoreList=[]
    # if we have less than 20 datapoints for a vid we skip over the
    # vid assuming something is missing in the raw data

    total = vid.iloc[:,4:20].values
    vidImPath=vid.iloc[:,0:2].values    

    if len(total) < time_steps :
        continue
    scoreVal=vid["Score"].values[0] + 1    
    max_total_for_vid = scoreVal.tolist()
    #max_total_for_vid = vid["Score"].values[0].tolist()

    for i in range(0,time_steps):
        videoName=vidImPath[i][0]
        imgName=vidImPath[i][1]
        path="./IMAGES/Train/"+videoName+"/"+imgName
        image = cv2.imread(path,0)
        hist = cv2.calcHist([image],[0],None,[num_bins],[0,256])

        hist_arr = hist.flatten()
        #img_arr = img_to_array(image)
        histValuesList.append(hist_arr)
        #scoreList.append(max_total_for_vid)

    trainX.append(histValuesList)
    trainY.append([max_total_for_vid])
    #trainY.append(scoreList)


print(len(trainX[0]))
#trainX = np.array([np.array(xi) for xi in trainX])

trainX=numpy.array(trainX)
trainY=numpy.array(trainY)

print(trainX.shape,trainY.shape)






vid_names_maps=[]


time_steps=19
# load the dataset
dataframe = pandas.read_csv('./Test.csv')
dataset = dataframe.values

#print(dataset)

# we group by day so we can process a video at a time.
grouped = dataframe.groupby(dataframe.VidName)

per_vid = []
for _, group in grouped:
    per_vid.append(group)



print(len(per_vid))




# generate sequences a vid at a time
for i,vid in enumerate(per_vid):
    histValuesList=[]
    scoreList=[]
    # if we have less than 20 datapoints for a vid we skip over the
    # vid assuming something is missing in the raw data

    total = vid.iloc[:,4:20].values

    vidImPath=vid.iloc[:,0:2].values


    if len(total)<time_steps :
        vid_names_maps.append()
        continue
    scoreVal=vid["Score"].values[0] + 1    
    max_total_for_vid = scoreVal.tolist()
    #max_total_for_vid = vid["Score"].values[0].tolist()


    for i in range(0,time_steps):
        #histValuesList.append(total[i])
        #print("Vid and Img name")
        #print(req[i][0],req[i][1])
        videoName=vidImPath[i][0]
        imgName=vidImPath[i][1]
        path="./IMAGES/Test/"+videoName+"/"+imgName
        image = cv2.imread(path,0)
        hist = cv2.calcHist([image],[0],None,[num_bins],[0,256])
        hist_arr = hist.flatten()

        histValuesList.append(hist_arr)
        #scoreList.append(max_total_for_vid)


    testX.append(histValuesList)
    testY.append([max_total_for_vid])
    #testY.append(scoreList)


print(len(testX[0]))
#trainX = np.array([np.array(xi) for xi in trainX])

testX=numpy.array(testX)
testY=numpy.array(testY)

print(testX.shape,testY.shape)


trainX=numpy.array(trainX)
trainX=trainX.reshape(-1,num_bins)
trainX=trainX.reshape(-1,19,num_bins)
print(numpy.max(trainX))


testX=numpy.array(testX)
testX=testX.reshape(-1,num_bins)
testX=testX.reshape(-1,19,num_bins)
print(numpy.max(testX))






trainX=numpy.array(trainX)
trainX=trainX.reshape(-1,num_bins)

trainX = trainX/numpy.max(trainX)

trainX=trainX.reshape(-1,19,num_bins)
print(trainX.shape,trainY.shape)




testX=numpy.array(testX)
testX=testX.reshape(-1,num_bins)

testX = testX/numpy.max(testX)

testX=testX.reshape(-1,19,num_bins)
print(testX.shape,testY.shape)




print(trainX.shape,trainY.shape)
print(testX.shape,testY.shape)
#print(valX.shape,valY.shape)







adam1 = optimizers.Adam(lr=0.001)
sgd1 = optimizers.SGD(lr=0.005) #0.005 or 6,100 neurons (1.24,1.12 with 0.003 and 0.2 )

print('Build model...')
# Build Model
#model = Sequential()  
model.add(LSTM(100, input_shape=(19, num_bins))) #100
model.add(Dense(1))
model.add(Dropout(0.1))


model.compile(loss='mse', optimizer=sgd1, metrics=['mse'])
#model.compile(loss='mse', optimizer=sgd1,metrics=['mean_squared_error'])
history =model.fit(trainX, trainY, nb_epoch=500, batch_size=20, verbose=2,shuffle=True) #500 batch =2     







# make predictions
trainPredict = model.predict(trainX)


trainScore = mean_squared_error(trainY, trainPredict)
print('Train Score: %.2f MSE' % (trainScore))




from keras import backend as K

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 

pred=model.predict(testX)

print(pred.shape)
print(testY.shape)



# calculate root mean squared error
#trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
#print('Train Score: %.2f RMSE' % (trainScore))
testScore = mean_squared_error(testY, pred)
print('Test Score: %.2f MSE' % (testScore))

#maeScore = root_mean_squared_error(testY, pred)
#print('RMSE Score: %.2f MAE' % (maeScore))

rmse = np.sqrt(((pred - testY) ** 2).mean(axis=0))
print('RMSE Score: %.2f rmse' % (rmse))


mae = mean_absolute_error(testY, pred)
print('MAE Score: %.2f mae' % (mae))
    
    
    
list1=[]
list2=[]
diff=[]
    
for i in range(0,len(pred)):

    print(testY[i],pred[i])
    list1.append(testY[i])
    list2.append(pred[i])
    diff.append(abs(testY[i]-pred[i]))
        
print(numpy.mean(diff))
stdVals=numpy.std(diff)
        
results_rmse.append(rmse)
results_mae.append(mae)
    
#stdVals = np.std(testY-pred)
print(stdVals)
results_std.append(stdVals)
    

117
19
(114, 19, 16) (114, 1)
55
19
(52, 19, 16) (52, 1)
174999.0
157653.0
(114, 19, 16) (114, 1)
(52, 19, 16) (52, 1)
(114, 19, 16) (114, 1)
(52, 19, 16) (52, 1)
Build model...


/Users/SindhuPhani/anaconda3/lib/python3.6/site-packages/keras/models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/500
 - 1s - loss: 3.2207 - mean_squared_error: 3.2207
Epoch 2/500
 - 0s - loss: 2.6435 - mean_squared_error: 2.6435
Epoch 3/500
 - 0s - loss: 2.2862 - mean_squared_error: 2.2862
Epoch 4/500
 - 0s - loss: 1.8720 - mean_squared_error: 1.8720
Epoch 5/500
 - 0s - loss: 1.7190 - mean_squared_error: 1.7190
Epoch 6/500
 - 0s - loss: 1.4600 - mean_squared_error: 1.4600
Epoch 7/500
 - 0s - loss: 1.6362 - mean_squared_error: 1.6362
Epoch 8/500
 - 0s - loss: 1.2653 - mean_squared_error: 1.2653
Epoch 9/500
 - 0s - loss: 1.3202 - mean_squared_error: 1.3202
Epoch 10/500
 - 0s - loss: 1.3635 - mean_squared_error: 1.3635
Epoch 11/500
 - 0s - loss: 1.2775 - mean_squared_error: 1.2775
Epoch 12/500
 - 0s - loss: 1.3327 - mean_squared_error: 1.3327
Epoch 13/500
 - 0s - loss: 1.2622 - mean_squared_error: 1.2622
Epoch 14/500
 - 0s - loss: 1.1279 - mean_squared_error: 1.1279
Epoch 15/500
 - 0s - loss: 1.1549 - mean_squared_error: 1.1549
Epoch 16/500
 - 0s - loss: 1.2331 - mean_squared_error: 1.2331
E

Epoch 131/500
 - 0s - loss: 1.2491 - mean_squared_error: 1.2491
Epoch 132/500
 - 0s - loss: 1.2365 - mean_squared_error: 1.2365
Epoch 133/500
 - 0s - loss: 1.1553 - mean_squared_error: 1.1553
Epoch 134/500
 - 0s - loss: 1.2845 - mean_squared_error: 1.2845
Epoch 135/500
 - 0s - loss: 1.0836 - mean_squared_error: 1.0836
Epoch 136/500
 - 0s - loss: 1.3284 - mean_squared_error: 1.3284
Epoch 137/500
 - 0s - loss: 1.0988 - mean_squared_error: 1.0988
Epoch 138/500
 - 0s - loss: 1.2013 - mean_squared_error: 1.2013
Epoch 139/500
 - 0s - loss: 1.2476 - mean_squared_error: 1.2476
Epoch 140/500
 - 0s - loss: 1.3626 - mean_squared_error: 1.3626
Epoch 141/500
 - 0s - loss: 1.2781 - mean_squared_error: 1.2781
Epoch 142/500
 - 0s - loss: 1.3488 - mean_squared_error: 1.3488
Epoch 143/500
 - 0s - loss: 1.3246 - mean_squared_error: 1.3246
Epoch 144/500
 - 0s - loss: 1.1699 - mean_squared_error: 1.1699
Epoch 145/500
 - 0s - loss: 1.1512 - mean_squared_error: 1.1512
Epoch 146/500
 - 0s - loss: 1.3803 - mea

 - 0s - loss: 1.3080 - mean_squared_error: 1.3080
Epoch 260/500
 - 0s - loss: 1.2767 - mean_squared_error: 1.2767
Epoch 261/500
 - 0s - loss: 1.4344 - mean_squared_error: 1.4344
Epoch 262/500
 - 0s - loss: 1.3346 - mean_squared_error: 1.3346
Epoch 263/500
 - 0s - loss: 1.0387 - mean_squared_error: 1.0387
Epoch 264/500
 - 0s - loss: 1.0292 - mean_squared_error: 1.0292
Epoch 265/500
 - 0s - loss: 1.1427 - mean_squared_error: 1.1427
Epoch 266/500
 - 0s - loss: 1.3348 - mean_squared_error: 1.3348
Epoch 267/500
 - 0s - loss: 1.0729 - mean_squared_error: 1.0729
Epoch 268/500
 - 0s - loss: 1.2399 - mean_squared_error: 1.2399
Epoch 269/500
 - 0s - loss: 1.1884 - mean_squared_error: 1.1884
Epoch 270/500
 - 0s - loss: 0.9849 - mean_squared_error: 0.9849
Epoch 271/500
 - 0s - loss: 1.2937 - mean_squared_error: 1.2937
Epoch 272/500
 - 0s - loss: 1.2386 - mean_squared_error: 1.2386
Epoch 273/500
 - 0s - loss: 1.2886 - mean_squared_error: 1.2886
Epoch 274/500
 - 0s - loss: 1.1508 - mean_squared_erro

Epoch 388/500
 - 0s - loss: 1.1510 - mean_squared_error: 1.1510
Epoch 389/500
 - 0s - loss: 1.1952 - mean_squared_error: 1.1952
Epoch 390/500
 - 0s - loss: 1.1039 - mean_squared_error: 1.1039
Epoch 391/500
 - 0s - loss: 0.9997 - mean_squared_error: 0.9997
Epoch 392/500
 - 0s - loss: 1.1008 - mean_squared_error: 1.1008
Epoch 393/500
 - 0s - loss: 1.3108 - mean_squared_error: 1.3108
Epoch 394/500
 - 0s - loss: 1.2806 - mean_squared_error: 1.2806
Epoch 395/500
 - 0s - loss: 1.2923 - mean_squared_error: 1.2923
Epoch 396/500
 - 0s - loss: 1.1562 - mean_squared_error: 1.1562
Epoch 397/500
 - 0s - loss: 1.1506 - mean_squared_error: 1.1506
Epoch 398/500
 - 0s - loss: 0.9783 - mean_squared_error: 0.9783
Epoch 399/500
 - 0s - loss: 1.1471 - mean_squared_error: 1.1471
Epoch 400/500
 - 0s - loss: 1.0877 - mean_squared_error: 1.0877
Epoch 401/500
 - 0s - loss: 1.0972 - mean_squared_error: 1.0972
Epoch 402/500
 - 0s - loss: 1.2007 - mean_squared_error: 1.2007
Epoch 403/500
 - 0s - loss: 1.1966 - mea

## Predictions on training data

In [18]:
# make predictions on training data
trainPredict = model.predict(trainX)


trainScore = mean_squared_error(trainY, trainPredict)
print('Train Score: %.2f MSE' % (trainScore))



rmse = np.sqrt(((trainPredict - trainY) ** 2).mean(axis=0))
print('RMSE Score: %.2f rmse' % (rmse))


mae = mean_absolute_error(trainY, trainPredict)
print('MAE Score: %.2f mae' % (mae))


    
list1=[]
list2=[]
diff=[]
    

for i in range(0,len(trainPredict)):
    print(trainY[i],trainPredict[i])
    list1.append(trainY[i])
    list2.append(trainPredict[i])
    diff.append(abs(trainY[i]-trainPredict[i]))
        
        
print(numpy.mean(diff))
stdVals=numpy.std(diff)
        
results_rmse.append(rmse)
results_mae.append(mae)
    
#stdVals = np.std(testY-pred)
print(stdVals)
results_std.append(stdVals)
    








Train Score: 0.95 MSE
RMSE Score: 0.97 rmse
MAE Score: 0.74 mae
[0] [0.8657829]
[0] [0.87993103]
[1] [1.3973285]
[4] [1.5053711]
[3] [1.2717153]
[3] [1.379099]
[2] [1.639708]
[2] [1.663112]
[2] [1.5343771]
[1] [1.4163165]
[2] [1.1997833]
[1] [1.0789253]
[4] [1.2343893]
[1] [0.9072609]
[4] [1.2738577]
[3] [1.1652279]
[0] [0.67907596]
[0] [0.790267]
[0] [0.9799684]
[2] [1.4247246]
[4] [1.3236446]
[1] [1.075547]
[1] [0.9333824]
[1] [1.370743]
[1] [1.5718254]
[1] [1.6793518]
[1] [0.88987446]
[2] [1.4067743]
[3] [1.6134661]
[1] [1.448578]
[1] [0.74892867]
[0] [0.6371925]
[3] [1.5682616]
[3] [1.563446]
[4] [1.6133012]
[3] [1.9407429]
[2] [1.9993695]
[2] [1.8576354]
[2] [1.0853736]
[2] [1.5776322]
[1] [1.2538174]
[1] [1.2942755]
[2] [1.3595243]
[2] [1.3074241]
[2] [1.049606]
[1] [0.74621785]
[0] [0.4598062]
[1] [0.9922364]
[0] [0.6146336]
[0] [0.36708295]
[1] [0.66955113]
[1] [1.0811235]
[1] [1.4658868]
[2] [1.6368428]
[2] [1.5438807]
[1] [1.7842182]
[2] [1.5640081]
[2] [1.1591837]
[3] [1.642

In [19]:
# trainY
list1 

[array([0]),
 array([0]),
 array([1]),
 array([4]),
 array([3]),
 array([3]),
 array([2]),
 array([2]),
 array([2]),
 array([1]),
 array([2]),
 array([1]),
 array([4]),
 array([1]),
 array([4]),
 array([3]),
 array([0]),
 array([0]),
 array([0]),
 array([2]),
 array([4]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([2]),
 array([3]),
 array([1]),
 array([1]),
 array([0]),
 array([3]),
 array([3]),
 array([4]),
 array([3]),
 array([2]),
 array([2]),
 array([2]),
 array([2]),
 array([1]),
 array([1]),
 array([2]),
 array([2]),
 array([2]),
 array([1]),
 array([0]),
 array([1]),
 array([0]),
 array([0]),
 array([1]),
 array([1]),
 array([1]),
 array([2]),
 array([2]),
 array([1]),
 array([2]),
 array([2]),
 array([3]),
 array([3]),
 array([3]),
 array([2]),
 array([3]),
 array([1]),
 array([1]),
 array([2]),
 array([2]),
 array([2]),
 array([1]),
 array([2]),
 array([1]),
 array([2]),
 array([2]),
 array([2]),
 array([0]),
 array([1]),
 array([2]),

In [20]:
# trainPredict
list2

[array([0.8657829], dtype=float32),
 array([0.87993103], dtype=float32),
 array([1.3973285], dtype=float32),
 array([1.5053711], dtype=float32),
 array([1.2717153], dtype=float32),
 array([1.379099], dtype=float32),
 array([1.639708], dtype=float32),
 array([1.663112], dtype=float32),
 array([1.5343771], dtype=float32),
 array([1.4163165], dtype=float32),
 array([1.1997833], dtype=float32),
 array([1.0789253], dtype=float32),
 array([1.2343893], dtype=float32),
 array([0.9072609], dtype=float32),
 array([1.2738577], dtype=float32),
 array([1.1652279], dtype=float32),
 array([0.67907596], dtype=float32),
 array([0.790267], dtype=float32),
 array([0.9799684], dtype=float32),
 array([1.4247246], dtype=float32),
 array([1.3236446], dtype=float32),
 array([1.075547], dtype=float32),
 array([0.9333824], dtype=float32),
 array([1.370743], dtype=float32),
 array([1.5718254], dtype=float32),
 array([1.6793518], dtype=float32),
 array([0.88987446], dtype=float32),
 array([1.4067743], dtype=float

## Predictions on test data

In [21]:

pred=model.predict(testX)

print(pred.shape)
print(testY.shape)


testScore = mean_squared_error(testY, pred)
print('Test Score: %.2f MSE' % (testScore))


rmse = np.sqrt(((pred - testY) ** 2).mean(axis=0))
print('RMSE Score: %.2f rmse' % (rmse))


mae = mean_absolute_error(testY, pred)
print('MAE Score: %.2f mae' % (mae))
    
    
    
list1=[]
list2=[]
diff=[]
    
for i in range(0,len(pred)):
    print(testY[i],pred[i])
    list1.append(testY[i])
    list2.append(pred[i])
    diff.append(abs(testY[i]-pred[i]))
        
print(numpy.mean(diff))
stdVals=numpy.std(diff)
        
results_rmse.append(rmse)
results_mae.append(mae)
    
#stdVals = np.std(testY-pred)
print(stdVals)
results_std.append(stdVals)
    

(52, 1)
(52, 1)
Test Score: 1.16 MSE
RMSE Score: 1.08 rmse
MAE Score: 0.90 mae
[0] [2.2048342]
[0] [1.1126734]
[0] [1.3997675]
[0] [0.5157764]
[1] [0.6071486]
[3] [2.0375755]
[0] [0.99859166]
[4] [1.7473166]
[0] [1.1756531]
[0] [1.6052155]
[3] [1.8819863]
[4] [1.265181]
[1] [1.3298031]
[2] [1.2512382]
[1] [1.0782294]
[2] [1.5557319]
[1] [1.3530238]
[2] [1.0890144]
[1] [1.8473151]
[0] [0.8526046]
[0] [1.0932784]
[0] [1.2102916]
[2] [1.2795185]
[0] [0.67705584]
[0] [0.6167114]
[0] [0.6345362]
[2] [0.8060204]
[1] [1.1223748]
[3] [1.4328588]
[0] [1.0436226]
[0] [0.7542808]
[0] [0.81572336]
[0] [1.2461239]
[2] [0.83412206]
[2] [1.0093734]
[1] [0.8660643]
[1] [2.1815915]
[2] [1.4881321]
[1] [1.6342332]
[2] [1.5071036]
[1] [1.5950464]
[1] [1.2154874]
[4] [1.5704111]
[1] [1.4999061]
[1] [0.95844793]
[2] [1.0333613]
[0] [0.82135975]
[1] [1.3967812]
[1] [1.0393807]
[1] [1.1608542]
[2] [0.669296]
[2] [0.7003008]
0.8969600854011682
0.5926745630259126


In [22]:
# testY
list1

[array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([1]),
 array([3]),
 array([0]),
 array([4]),
 array([0]),
 array([0]),
 array([3]),
 array([4]),
 array([1]),
 array([2]),
 array([1]),
 array([2]),
 array([1]),
 array([2]),
 array([1]),
 array([0]),
 array([0]),
 array([0]),
 array([2]),
 array([0]),
 array([0]),
 array([0]),
 array([2]),
 array([1]),
 array([3]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([2]),
 array([2]),
 array([1]),
 array([1]),
 array([2]),
 array([1]),
 array([2]),
 array([1]),
 array([1]),
 array([4]),
 array([1]),
 array([1]),
 array([2]),
 array([0]),
 array([1]),
 array([1]),
 array([1]),
 array([2]),
 array([2])]

In [23]:
# pred
list2

[array([2.2048342], dtype=float32),
 array([1.1126734], dtype=float32),
 array([1.3997675], dtype=float32),
 array([0.5157764], dtype=float32),
 array([0.6071486], dtype=float32),
 array([2.0375755], dtype=float32),
 array([0.99859166], dtype=float32),
 array([1.7473166], dtype=float32),
 array([1.1756531], dtype=float32),
 array([1.6052155], dtype=float32),
 array([1.8819863], dtype=float32),
 array([1.265181], dtype=float32),
 array([1.3298031], dtype=float32),
 array([1.2512382], dtype=float32),
 array([1.0782294], dtype=float32),
 array([1.5557319], dtype=float32),
 array([1.3530238], dtype=float32),
 array([1.0890144], dtype=float32),
 array([1.8473151], dtype=float32),
 array([0.8526046], dtype=float32),
 array([1.0932784], dtype=float32),
 array([1.2102916], dtype=float32),
 array([1.2795185], dtype=float32),
 array([0.67705584], dtype=float32),
 array([0.6167114], dtype=float32),
 array([0.6345362], dtype=float32),
 array([0.8060204], dtype=float32),
 array([1.1223748], dtype=f

In [24]:
# abs(testY-pred)
diff

[array([2.20483422]),
 array([1.1126734]),
 array([1.39976752]),
 array([0.5157764]),
 array([0.39285141]),
 array([0.96242452]),
 array([0.99859166]),
 array([2.2526834]),
 array([1.1756531]),
 array([1.60521555]),
 array([1.11801374]),
 array([2.73481905]),
 array([0.32980311]),
 array([0.74876177]),
 array([0.07822943]),
 array([0.44426811]),
 array([0.35302377]),
 array([0.91098559]),
 array([0.84731507]),
 array([0.85260463]),
 array([1.09327841]),
 array([1.21029162]),
 array([0.72048151]),
 array([0.67705584]),
 array([0.61671138]),
 array([0.63453621]),
 array([1.19397962]),
 array([0.12237477]),
 array([1.56714118]),
 array([1.04362261]),
 array([0.75428081]),
 array([0.81572336]),
 array([1.24612391]),
 array([1.16587794]),
 array([0.99062657]),
 array([0.13393569]),
 array([1.18159151]),
 array([0.51186788]),
 array([0.63423324]),
 array([0.49289644]),
 array([0.5950464]),
 array([0.21548736]),
 array([2.42958891]),
 array([0.49990606]),
 array([0.04155207]),
 array([0.96663

## Finding the index of success and failure videos

In [31]:
success=[]
failure=[]
for i in range(0,len(diff)):
    if diff[i]<0.8:
        success.append(i)
    else:
        failure.append(i)

In [32]:
success

[3,
 4,
 12,
 13,
 14,
 15,
 16,
 22,
 23,
 24,
 25,
 27,
 30,
 35,
 37,
 38,
 39,
 40,
 41,
 43,
 44,
 47,
 48,
 49]

In [33]:
failure

[0,
 1,
 2,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 17,
 18,
 19,
 20,
 21,
 26,
 28,
 29,
 31,
 32,
 33,
 34,
 36,
 42,
 45,
 46,
 50,
 51]

In [46]:
print(len(success))
print(len(failure))

24
28


# Identifying success videos

In [71]:
testX=[]
testY=[]

goodVideos=[]

numpy.random.seed(3)
time_steps=19
# load the dataset
dataframe = pandas.read_csv('./Test.csv')
dataset = dataframe.values

#print(dataset)

# we group by day so we can process a video at a time.
grouped = dataframe.groupby(dataframe.VidName)

per_vid = []
for _, group in grouped:
    per_vid.append(group)



print(len(per_vid))




# generate sequences a vid at a time
for i,vid in enumerate(per_vid):
    
        histValuesList=[]
        scoreList=[]
        # if we have less than 20 datapoints for a vid we skip over the
        # vid assuming something is missing in the raw data

        total = vid.iloc[:,4:20].values

        vidImPath=vid.iloc[:,0:2].values


        if len(total)<time_steps :
            print(i)
            continue
        scoreVal=vid["Score"].values[0] + 1    
        max_total_for_vid = scoreVal.tolist()
        goodVideos.append(vid["VidName"].values[0])
        #max_total_for_vid = vid["Score"].values[0].tolist()


        for i in range(0,time_steps):
            #histValuesList.append(total[i])
            #print("Vid and Img name")
            #print(req[i][0],req[i][1])
            videoName=vidImPath[i][0]
            imgName=vidImPath[i][1]
            path="./IMAGES/Test/"+videoName+"/"+imgName
            image = cv2.imread(path,0)
            hist = cv2.calcHist([image],[0],None,[num_bins],[0,256])
            hist_arr = hist.flatten()

            histValuesList.append(hist_arr)
            #scoreList.append(max_total_for_vid)


        testX.append(histValuesList)
        testY.append([max_total_for_vid])
        #testY.append(scoreList)
    

print(len(testX[0]))
#trainX = np.array([np.array(xi) for xi in trainX])

testX=numpy.array(testX)
testY=numpy.array(testY)

print(testX.shape,testY.shape)



testX=numpy.array(testX)
testX=testX.reshape(-1,num_bins)

testX = testX/numpy.max(testX)

testX=testX.reshape(-1,19,num_bins)
print(testX.shape,testY.shape)



print(testX.shape,testY.shape)





55
0
10
54
19
(52, 19, 16) (52, 1)
(52, 19, 16) (52, 1)
(52, 19, 16) (52, 1)


In [72]:
# Videos considered after preprocessing
goodVideos

['GRMN0002',
 'GRMN0003',
 'GRMN0004',
 'GRMN0005',
 'GRMN0006',
 'GRMN0007',
 'GRMN0008',
 'GRMN0009',
 'GRMN0010',
 'GRMN0012',
 'GRMN0013',
 'GRMN0014',
 'GRMN0015',
 'GRMN0016',
 'GRMN0017',
 'GRMN0018',
 'GRMN0019',
 'GRMN0020',
 'GRMN0021',
 'GRMN0022',
 'GRMN0023',
 'GRMN0024',
 'GRMN0025',
 'GRMN0026',
 'GRMN0027',
 'GRMN0028',
 'GRMN0029',
 'GRMN0030',
 'GRMN0031',
 'GRMN0032',
 'GRMN0033',
 'GRMN0034',
 'GRMN0035',
 'GRMN0036',
 'GRMN0037',
 'GRMN0038',
 'GRMN0039',
 'GRMN0040',
 'GRMN0041',
 'GRMN0042',
 'GRMN0043',
 'GRMN0044',
 'GRMN0045',
 'GRMN0046',
 'GRMN0047',
 'GRMN0048',
 'GRMN0049',
 'GRMN0050',
 'GRMN0051',
 'GRMN0052',
 'GRMN0053',
 'GRMN0054']

In [73]:
print(len(goodVideos))

52


In [74]:
sucX=[]
sucName=[]
failX=[]
failName=[]
sucY=[]
failY=[]
for i in range(0,len(testX)):
    if i in success:
        sucX.append(testX[i])
        sucY.append(testY[i])
        sucName.append(goodVideos[i])
    elif i in failure:
        failX.append(testX[i])
        failY.append(testY[i])
        failName.append(goodVideos[i])
    
print(len(sucX),len(failX)) 
print(len(sucY),len(failY))   

24 28
24 28


In [75]:
print(len(sucX[0]))
#trainX = np.array([np.array(xi) for xi in trainX])

sucX=numpy.array(sucX)
sucY=numpy.array(sucY)

print(sucX.shape,sucY.shape)



sucX=numpy.array(sucX)
sucX=sucX.reshape(-1,num_bins)

sucX = sucX/numpy.max(sucX)

sucX=sucX.reshape(-1,19,num_bins)
print(sucX.shape,sucY.shape)



print(sucX.shape,sucY.shape)


19
(24, 19, 16) (24, 1)
(24, 19, 16) (24, 1)
(24, 19, 16) (24, 1)


# Successful cases (VideoName,GroundTruth,Predictions)

In [76]:

pred=model.predict(sucX)
print(pred.shape)
print(sucX.shape)



    
for i in range(0,len(pred)):
    print(sucName[i],sucY[i],pred[i])
    

(24, 1)
(24, 19, 16)
GRMN0005 [0] [0.5157764]
GRMN0006 [1] [0.6071486]
GRMN0015 [1] [1.3298031]
GRMN0016 [2] [1.2512382]
GRMN0017 [1] [1.0782294]
GRMN0018 [2] [1.5557319]
GRMN0019 [1] [1.3530238]
GRMN0025 [2] [1.2795185]
GRMN0026 [0] [0.67705584]
GRMN0027 [0] [0.6167114]
GRMN0028 [0] [0.6345362]
GRMN0030 [1] [1.1223748]
GRMN0033 [0] [0.7542808]
GRMN0038 [1] [0.8660643]
GRMN0040 [2] [1.4881321]
GRMN0041 [1] [1.6342332]
GRMN0042 [2] [1.5071036]
GRMN0043 [1] [1.5950464]
GRMN0044 [1] [1.2154874]
GRMN0046 [1] [1.4999061]
GRMN0047 [1] [0.95844793]
GRMN0050 [1] [1.3967812]
GRMN0051 [1] [1.0393807]
GRMN0052 [1] [1.1608542]


# Identifying failure videos

In [77]:
print(len(failX[0]))
#trainX = np.array([np.array(xi) for xi in trainX])

failX=numpy.array(failX)
failY=numpy.array(failY)

print(failX.shape,failY.shape)



failX=numpy.array(failX)
failX=failX.reshape(-1,num_bins)

failX = failX/numpy.max(failX)

failX=failX.reshape(-1,19,num_bins)
print(failX.shape,failY.shape)



print(failX.shape,failY.shape)


19
(28, 19, 16) (28, 1)
(28, 19, 16) (28, 1)
(28, 19, 16) (28, 1)


# Failure cases (VideoName,GroundTruth,Predictions)

In [78]:

pred=model.predict(failX)
print(pred.shape)
print(failX.shape)



    
for i in range(0,len(pred)):
    print(failName[i],failY[i],pred[i])
    

(28, 1)
(28, 19, 16)
GRMN0002 [0] [2.4318638]
GRMN0003 [0] [1.1863565]
GRMN0004 [0] [1.5234164]
GRMN0007 [3] [2.2406538]
GRMN0008 [0] [1.0488473]
GRMN0009 [4] [1.9225577]
GRMN0010 [0] [1.2607315]
GRMN0012 [0] [1.7631961]
GRMN0013 [3] [2.0698028]
GRMN0014 [4] [1.3673921]
GRMN0020 [2] [1.1583776]
GRMN0021 [1] [2.0517538]
GRMN0022 [0] [0.8708576]
GRMN0023 [0] [1.1618351]
GRMN0024 [0] [1.3017342]
GRMN0029 [2] [0.814874]
GRMN0031 [3] [1.5692872]
GRMN0032 [0] [1.1031232]
GRMN0034 [0] [0.82694626]
GRMN0035 [0] [1.345525]
GRMN0036 [2] [0.84939003]
GRMN0037 [2] [1.0610719]
GRMN0039 [1] [2.4250908]
GRMN0045 [4] [1.7261335]
GRMN0048 [2] [1.0891594]
GRMN0049 [0] [0.8375466]
GRMN0053 [2] [0.6545784]
GRMN0054 [2] [0.6878667]


# Preparing data for maps

## Preparing training data

In [1]:
import os  
import numpy as np  
import pandas as pd  
import matplotlib
import matplotlib.pyplot as plt  
%matplotlib inline
from scipy import misc
import matplotlib.image as mpimg
import csv  
import sys
from collections import defaultdict
import sys
import re
import time
import random
import cv2
from pylab import imread, imshow, gray, mean
gray()



# column names same as in csv file
col_names = ['VidName','ImgName','Lati','Longi']









np.random.seed(7)
time_steps=19
# load the dataset

# path of the video file csv with lat and longi score values

video_scores_path = "./Trainold.csv"    
dataframe = pd.read_csv(video_scores_path)

dataset = dataframe.values



# we group by day so we can process a video at a time.
grouped = dataframe.groupby(dataframe.VidName)

per_vid = []
for _, group in grouped:
    per_vid.append(group)
    
    

print(len(per_vid))



trainX=[]
trainY=[]

# generate sequences a vid at a time
for i,vid in enumerate(per_vid):
    histValuesList=[]
    # if we have less than 20 datapoints for a vid we skip over the
    # vid assuming something is missing in the raw data
    
    total = vid[col_names].values
    
    
    if len(total) < time_steps :
        continue
    scoreVal=vid["Score"].values[0] + 1    
    max_total_for_vid = scoreVal.tolist()
    #max_total_for_vid = vid["Score"].values[0].tolist()
    
   
    histValuesList.append(total[1])
    trainX.append(histValuesList)
    #trainX.append(total[0:time_steps+1])
    trainY.append([max_total_for_vid])


print(len(trainX[0]))
#trainX = np.array([np.array(xi) for xi in trainX])

trainX=np.array(trainX)
trainY=np.array(trainY)

print(trainX.shape,trainY.shape)


117
1
(114, 1, 4) (114, 1)


In [2]:

prediction=[[0,0.8657829],[0,0.87993103],[1,1.3973285],[4,1.5053711],[3,1.2717153],[3,1.379099],[2,1.639708],[2,1.663112],[2,1.5343771],[1,1.4163165],[2,1.1997833],[1,1.0789253],[4,1.2343893],[1,0.9072609],[4,1.2738577],[3,1.1652279],[0,0.67907596],[0,0.790267],[0,0.9799684],[2,1.4247246],[4,1.3236446],[1,1.075547],[1,0.9333824],[1,1.370743],[1,1.5718254],[1,1.6793518],[1,0.88987446],[2,1.4067743],[3,1.6134661],[1,1.448578],[1,0.74892867],[0,0.6371925],[3,1.5682616],[3,1.563446],[4,1.6133012],[3,1.9407429],[2,1.9993695],[2,1.8576354],[2,1.0853736],[2,1.5776322],[1,1.2538174],[1,1.2942755],[2,1.3595243],[2,1.3074241],[2,1.049606],[1,0.74621785],[0,0.4598062],[1,0.9922364],[0,0.6146336],[0,0.36708295],[1,0.66955113],[1,1.0811235],[1,1.4658868],[2,1.6368428],[2,1.5438807],[1,1.7842182],[2,1.5640081],[2,1.1591837],[3,1.6423346],[3,1.9664043],[3,1.7660748],[2,1.1584783],[3,0.9957206],[1,0.88221407],[1,1.4206612],[2,1.1734943],[2,1.3031573],[2,1.2647648],[1,0.6225684],[2,0.9201188],[1,0.8307609],[2,1.2336228],[2,1.3905258],[2,1.1744916],[0,1.4268484],[1,1.4361352],[2,1.1651131],[3,0.8364122],[2,0.62921],[1,1.1430522],[1,0.97430265],[1,0.7059832],[1,0.8587964],[1,0.8164649],[1,0.70572674],[1,0.8964597],[1,1.0888579],[0,1.0548699],[2,0.99313796],[0,0.70063215],[1,1.3648108],[2,1.1785933],[2,1.0186093],[3,1.1419646],[1,1.0615672],[1,1.0875626],[1,0.9385246],[1,1.2282915],[1,1.3046808],[0,1.159186],[1,1.2628542],[0,1.0115659],[1,1.6899377],[1,1.5836076],[3,1.0448731],[2,0.74293053],[2,0.7788062],[1,1.1646601],[0,0.8948417],[2,1.330603],[1,1.1767646],[1,1.1695198],[2,1.0223768],[0,0.73476326]]   



truth=[]
pred=[]
for item in zip(trainX,prediction):
    #print(item[0][0][2],item[1][0])
    truthTemp=[]
    predTemp=[]
    truthTemp.append(item[0][0][2])
    truthTemp.append(item[0][0][3])
    truthTemp.append(item[1][0])
    
    predTemp.append(item[0][0][2])
    predTemp.append(item[0][0][3])
    predTemp.append(item[1][1])
    truth.append(truthTemp)
    pred.append(predTemp)
    
truth
print("pred")
truth




pred


[[-1.0, -1.0, 0],
 [-1.0, -1.0, 0],
 [-1.0, -1.0, 1],
 [37.394, -121.945, 4],
 [37.3893, -121.9555, 3],
 [37.383, -121.9646, 3],
 [37.3871, -121.98141000000001, 2],
 [37.3927, -122.0008, 2],
 [37.3955, -122.01299999999999, 2],
 [37.4, -122.03, 1],
 [37.397, -122.04, 2],
 [37.39, -122.05, 1],
 [37.3884, -122.0627, 4],
 [37.3872, -122.057, 1],
 [37.384, -122.0528, 4],
 [37.3776, -122.0551, 3],
 [37.374, -122.0551, 0],
 [37.376, -122.0554, 0],
 [37.376, -122.0554, 0],
 [37.38, -122.0572, 2],
 [37.388000000000005, -122.0572, 4],
 [37.3923, -122.051, 1],
 [37.4025, -122.0283, 1],
 [37.4076, -122.0056, 1],
 [37.4106, -122.9911, 1],
 [37.4164, -122.9745, 1],
 [37.4187, -122.956, 1],
 [37.415, -122.954, 2],
 [37.4108, -122.9469, 3],
 [37.4061, -121.94200000000001, 1],
 [37.403, -121.943, 1],
 [37.403, -121.943, 0],
 [37.39, -121.954, 3],
 [37.38, -121.96, 3],
 [37.378, -121.96600000000001, 4],
 [37.37, -121.98, 3],
 [37.3776, -121.9977, 2],
 [37.3778, -122.01100000000001, 2],
 [37.3762, -122.0

In [3]:
# visualizing ground truth first

tskip=[]
tnew=[]
tslworn=[]
tworn=[]
theavy=[]

for elem in truth:
    if(elem[0]==0 or elem[1]==0):
        continue
    
    if(elem[2]==0):
        tempL=[]
        tempL.append(elem[0])
        tempL.append(elem[1])
        tskip.append(tempL)
    elif(elem[2]==1):
        tempL=[]
        tempL.append(elem[0])
        tempL.append(elem[1])
        tnew.append(tempL)
    elif(elem[2]==2):
        tempL=[]
        tempL.append(elem[0])
        tempL.append(elem[1])
        tslworn.append(tempL)
    elif(elem[2]==3):
        tempL=[]
        tempL.append(elem[0])
        tempL.append(elem[1])
        tworn.append(tempL)
    elif(elem[2]==4):
        tempL=[]
        tempL.append(elem[0])
        tempL.append(elem[1])
        theavy.append(tempL)
        
        
    

In [4]:
print(tskip)
lat0=[]
long0=[]

for item in tskip:
    print(item[0],item[1])
    lat0.append(item[0])
    
    long0.append(item[0])
    
    
    
lat0=[]
long0=[]

for item in tskip:
    lat0.append(item[0])
    long0.append(item[1])

print(lat0)
print(long0)

[[-1.0, -1.0], [-1.0, -1.0], [37.374, -122.0551], [37.376, -122.0554], [37.376, -122.0554], [37.403, -121.943], [37.352, -122.00953999999999], [37.351, -122.0054], [37.3516, -122.0053], [37.400999999999996, -121.939], [37.396, -122.014], [37.402, -122.02600000000001], [37.3962, -122.0114], [37.402, -122.02600000000001], [37.384, -121.971], [37.4015, -122.027]]
-1.0 -1.0
-1.0 -1.0
37.374 -122.0551
37.376 -122.0554
37.376 -122.0554
37.403 -121.943
37.352 -122.00953999999999
37.351 -122.0054
37.3516 -122.0053
37.400999999999996 -121.939
37.396 -122.014
37.402 -122.02600000000001
37.3962 -122.0114
37.402 -122.02600000000001
37.384 -121.971
37.4015 -122.027
[-1.0, -1.0, 37.374, 37.376, 37.376, 37.403, 37.352, 37.351, 37.3516, 37.400999999999996, 37.396, 37.402, 37.3962, 37.402, 37.384, 37.4015]
[-1.0, -1.0, -122.0551, -122.0554, -122.0554, -121.943, -122.00953999999999, -122.0054, -122.0053, -121.939, -122.014, -122.02600000000001, -122.0114, -122.02600000000001, -121.971, -122.027]


# Visualizing Training data ground truth on google maps.

#### Please replace plot.api_key with your own key

In [7]:
# ADDING ONE AT A TIME
# ['#440154', '#404387', '#29788E', '#22A784', '#79D151', '#FDE724']

from bokeh.io import output_file, output_notebook, show
from bokeh.models import (
  GMapPlot, GMapOptions, ColumnDataSource, Circle, LogColorMapper, BasicTicker, ColorBar,
    DataRange1d, PanTool, WheelZoomTool, BoxSelectTool
)
from bokeh.models.mappers import ColorMapper, LinearColorMapper
from bokeh.palettes import Viridis5



from bokeh.io import output_file, show
from bokeh.models import (
  GMapPlot, GMapOptions, ColumnDataSource, Circle, Range1d, PanTool, WheelZoomTool, BoxSelectTool
)

map_options = GMapOptions(lat=37.5324, lng=-121.9687, map_type="roadmap", zoom=11)

plot = GMapPlot(x_range=Range1d(), y_range=Range1d(), map_options=map_options)
plot.title.text = "San Francisco Bay Area"

# For GMaps to function, Google requires you obtain and enable an API key:
#
#     https://developers.google.com/maps/documentation/javascript/get-api-key
#
# Replace the value below with your personal API key:

plot.api_key = "key"

lat0=[]
long0=[]

for item in tskip:
    lat0.append(item[0])
    long0.append(item[1])


source = ColumnDataSource(
    data=dict(
        lat=lat0,
        lon=long0,
    )
)

circle = Circle(x="lon", y="lat", size=6, fill_color="black", fill_alpha=1.8, line_color=None)
plot.add_glyph(source, circle)
#output_notebook()
#plot.add_tools(PanTool(), WheelZoomTool(), BoxSelectTool())
#output_file("gmap_plot.html")
#show(plot)


# *****************************************************************************************************

# # FOR NEW ROAD

lat1=[]
long1=[]

for item in tnew:
    lat1.append(item[0])
    long1.append(item[1])


source1 = ColumnDataSource(
    data=dict(
        lat1=lat1,
        lon1=long1,
    )
)

circle1 = Circle(x="lon1", y="lat1", size=16, fill_color="green", fill_alpha=1.8, line_color=None)
plot.add_glyph(source1, circle1)
#output_notebook()
#plot.add_tools(PanTool(), WheelZoomTool(), BoxSelectTool())
#output_file("gmap_plot.html")
#show(plot)




# *****************************************************************************************************

# # FOR SLIGHTLY WORN ROAD

lat2=[]
long2=[]

for item in tslworn:
    lat2.append(item[0])
    long2.append(item[1])


source2 = ColumnDataSource(
    data=dict(
        lat2=lat2,
        lon2=long2,
    )
)

circle2 = Circle(x="lon2", y="lat2", size=13, fill_color="yellow", fill_alpha=1.8, line_color=None)
plot.add_glyph(source2, circle2)
#output_notebook()
#plot.add_tools(PanTool(), WheelZoomTool(), BoxSelectTool())
#output_file("gmap_plot.html")
#show(plot)






# *****************************************************************************************************

# # FOR  WORN ROAD

lat3=[]
long3=[]

for item in tworn:
    lat3.append(item[0])
    long3.append(item[1])


source3 = ColumnDataSource(
    data=dict(
        lat3=lat3,
        lon3=long3,
    )
)

circle3 = Circle(x="lon3", y="lat3", size=10, fill_color="purple", fill_alpha=1.8, line_color=None)
plot.add_glyph(source3, circle3)
#output_notebook()
#plot.add_tools(PanTool(), WheelZoomTool(), BoxSelectTool())
#output_file("gmap_plot.html")
#show(plot)









# *****************************************************************************************************

# # FOR  HEAVY ROAD

lat4=[]
long4=[]

for item in theavy:
    lat4.append(item[0])
    long4.append(item[1])


source4 = ColumnDataSource(
    data=dict(
        lat4=lat4,
        lon4=long4,
    )
)

circle4 = Circle(x="lon4", y="lat4", size=8, fill_color="red", fill_alpha=1.8, line_color=None)
plot.add_glyph(source4, circle4)
output_notebook()
plot.add_tools(PanTool(), WheelZoomTool(), BoxSelectTool())
#output_file("gmap_plot.html")
show(plot)



Loading BokehJS ...

In [8]:
pred

[[-1.0, -1.0, 0.8657829],
 [-1.0, -1.0, 0.87993103],
 [-1.0, -1.0, 1.3973285],
 [37.394, -121.945, 1.5053711],
 [37.3893, -121.9555, 1.2717153],
 [37.383, -121.9646, 1.379099],
 [37.3871, -121.98141000000001, 1.639708],
 [37.3927, -122.0008, 1.663112],
 [37.3955, -122.01299999999999, 1.5343771],
 [37.4, -122.03, 1.4163165],
 [37.397, -122.04, 1.1997833],
 [37.39, -122.05, 1.0789253],
 [37.3884, -122.0627, 1.2343893],
 [37.3872, -122.057, 0.9072609],
 [37.384, -122.0528, 1.2738577],
 [37.3776, -122.0551, 1.1652279],
 [37.374, -122.0551, 0.67907596],
 [37.376, -122.0554, 0.790267],
 [37.376, -122.0554, 0.9799684],
 [37.38, -122.0572, 1.4247246],
 [37.388000000000005, -122.0572, 1.3236446],
 [37.3923, -122.051, 1.075547],
 [37.4025, -122.0283, 0.9333824],
 [37.4076, -122.0056, 1.370743],
 [37.4106, -122.9911, 1.5718254],
 [37.4164, -122.9745, 1.6793518],
 [37.4187, -122.956, 0.88987446],
 [37.415, -122.954, 1.4067743],
 [37.4108, -122.9469, 1.6134661],
 [37.4061, -121.94200000000001, 1.44

In [9]:
from bokeh.io import output_file, output_notebook, show
from bokeh.models import (
  GMapPlot, GMapOptions, ColumnDataSource, Circle, LogColorMapper, BasicTicker, ColorBar,
    DataRange1d, PanTool, WheelZoomTool, BoxSelectTool
)
from bokeh.models.mappers import ColorMapper, LinearColorMapper
from bokeh.palettes import Viridis5



from bokeh.io import output_file, show
from bokeh.models import (
  GMapPlot, GMapOptions, ColumnDataSource, Circle, Range1d, PanTool, WheelZoomTool, BoxSelectTool
)

# visualizing predictions

pskip=[]
pnew=[]
pslworn=[]
pworn=[]
pheavy=[]

for elem in pred:
    if(elem[0]==0 or elem[1]==0):
        continue
    
    if(elem[2]>=0 and elem[2]<=0.5):
        tempL=[]
        tempL.append(elem[0])
        tempL.append(elem[1])
        pskip.append(tempL)
    elif(elem[2]>0.5 and elem[2]<=1.5):
        tempL=[]
        tempL.append(elem[0])
        tempL.append(elem[1])
        pnew.append(tempL)
    elif(elem[2]>1.5 and elem[2]<=2.5):
        tempL=[]
        tempL.append(elem[0])
        tempL.append(elem[1])
        pslworn.append(tempL)
    elif(elem[2]>2.5 and elem[2]<=3.5):
        tempL=[]
        tempL.append(elem[0])
        tempL.append(elem[1])
        pworn.append(tempL)
    elif(elem[2]>3.5):
        tempL=[]
        tempL.append(elem[0])
        tempL.append(elem[1])
        pheavy.append(tempL)
        
        

        


# Visualizing Training data predictions on google maps.

#### Please replace plot.api_key with your own key

In [10]:
# predictions plot_pred
# ADDING ONE AT A TIME
# ['#440154', '#404387', '#29788E', '#22A784', '#79D151', '#FDE724']



        
        
        
        
        
        
        
        
        
        


map_options = GMapOptions(lat=37.5324, lng=-121.9687, map_type="roadmap", zoom=11)

plot_pred = GMapPlot(x_range=Range1d(), y_range=Range1d(), map_options=map_options)
plot_pred.title.text = "San Francisco Bay Area"

# For GMaps to function, Google requires you obtain and enable an API key:
#
#     https://developers.google.com/maps/documentation/javascript/get-api-key
#
# Replace the value below with your personal API key:

plot_pred.api_key = "key"

lat0=[]
long0=[]

for item in pskip:
    lat0.append(item[0])
    long0.append(item[1])


source = ColumnDataSource(
    data=dict(
        lat=lat0,
        lon=long0,
    )
)

circle = Circle(x="lon", y="lat", size=6, fill_color="black", fill_alpha=1.8, line_color=None)
plot_pred.add_glyph(source, circle)
#output_notebook()
#plot_pred.add_tools(PanTool(), WheelZoomTool(), BoxSelectTool())
#output_file("gmap_plot_pred.html")
#show(plot_pred)


# *****************************************************************************************************

# # FOR NEW ROAD

lat1=[]
long1=[]

for item in pnew:
    lat1.append(item[0])
    long1.append(item[1])


source1 = ColumnDataSource(
    data=dict(
        lat1=lat1,
        lon1=long1,
    )
)

circle1 = Circle(x="lon1", y="lat1", size=16, fill_color="green", fill_alpha=1.8, line_color=None)
plot_pred.add_glyph(source1, circle1)
#output_notebook()
#plot_pred.add_tools(PanTool(), WheelZoomTool(), BoxSelectTool())
#output_file("gmap_plot_pred.html")
#show(plot_pred)




# *****************************************************************************************************

# # FOR SLIGHTLY WORN ROAD

lat2=[]
long2=[]

for item in pslworn:
    lat2.append(item[0])
    long2.append(item[1])


source2 = ColumnDataSource(
    data=dict(
        lat2=lat2,
        lon2=long2,
    )
)

circle2 = Circle(x="lon2", y="lat2", size=13, fill_color="yellow", fill_alpha=1.8, line_color=None)
plot_pred.add_glyph(source2, circle2)
#output_notebook()
#plot_pred.add_tools(PanTool(), WheelZoomTool(), BoxSelectTool())
#output_file("gmap_plot_pred.html")
#show(plot_pred)






# *****************************************************************************************************

# # FOR  WORN ROAD

lat3=[]
long3=[]

for item in pworn:
    lat3.append(item[0])
    long3.append(item[1])


source3 = ColumnDataSource(
    data=dict(
        lat3=lat3,
        lon3=long3,
    )
)

circle3 = Circle(x="lon3", y="lat3", size=10, fill_color="purple", fill_alpha=1.8, line_color=None)
plot_pred.add_glyph(source3, circle3)
#output_notebook()
#plot_pred.add_tools(PanTool(), WheelZoomTool(), BoxSelectTool())
#output_file("gmap_plot_pred.html")
#show(plot_pred)









# *****************************************************************************************************

# # FOR  HEAVY ROAD

lat4=[]
long4=[]

for item in pheavy:
    lat4.append(item[0])
    long4.append(item[1])


source4 = ColumnDataSource(
    data=dict(
        lat4=lat4,
        lon4=long4,
    )
)

circle4 = Circle(x="lon4", y="lat4", size=8, fill_color="red", fill_alpha=1.8, line_color=None)
plot_pred.add_glyph(source4, circle4)
output_notebook()
plot_pred.add_tools(PanTool(), WheelZoomTool(), BoxSelectTool())
#output_file("gmap_plot_pred.html")
show(plot_pred)


        



Loading BokehJS ...

## Preparing test data

In [11]:
import os  
import numpy as np  
import pandas as pd  
import matplotlib
import matplotlib.pyplot as plt  
%matplotlib inline
from scipy import misc
import matplotlib.image as mpimg
import csv  
import sys
from collections import defaultdict
import sys
import re
import time
import random
import cv2
from pylab import imread, imshow, gray, mean
gray()



# column names same as in csv file
col_names = ['VidName','ImgName','Lati','Longi']









np.random.seed(7)
time_steps=19
# load the dataset

# path of the video file csv with lat and longi score values

video_scores_path = "./Test.csv"    
dataframe = pd.read_csv(video_scores_path)

dataset = dataframe.values



# we group by day so we can process a video at a time.
grouped = dataframe.groupby(dataframe.VidName)

per_vid = []
for _, group in grouped:
    per_vid.append(group)
    
    

print(len(per_vid))



trainX=[]
trainY=[]

# generate sequences a vid at a time
for i,vid in enumerate(per_vid):
    histValuesList=[]
    # if we have less than 20 datapoints for a vid we skip over the
    # vid assuming something is missing in the raw data
    
    total = vid[col_names].values
    
    
    if len(total) < time_steps :
        continue
    scoreVal=vid["Score"].values[0] + 1    
    max_total_for_vid = scoreVal.tolist()
    #max_total_for_vid = vid["Score"].values[0].tolist()
    
   
    histValuesList.append(total[1])
    trainX.append(histValuesList)
    #trainX.append(total[0:time_steps+1])
    trainY.append([max_total_for_vid])


print(len(trainX[0]))
#trainX = np.array([np.array(xi) for xi in trainX])

trainX=np.array(trainX)
trainY=np.array(trainY)

print(trainX.shape,trainY.shape)


55
1
(52, 1, 4) (52, 1)


In [12]:
prediction=[[0,2.2048342],
[0,1.1126734],
[0,1.3997675],
[0,0.5157764],
[1,0.6071486],
[3,2.0375755],
[0,0.99859166],
[4,1.7473166],
[0,1.1756531],
[0,1.6052155],
[3,1.8819863],
[4,1.265181],
[1,1.3298031],
[2,1.2512382],
[1,1.0782294],
[2,1.5557319],
[1,1.3530238],
[2,1.0890144],
[1,1.8473151],
[0,0.8526046],
[0,1.0932784],
[0,1.2102916],
[2,1.2795185],
[0,0.67705584],
[0,0.6167114],
[0,0.6345362],
[2,0.8060204],
[1,1.1223748],
[3,1.4328588],
[0,1.0436226],
[0,0.7542808],
[0,0.81572336],
[0,1.2461239],
[2,0.83412206],
[2,1.0093734],
[1,0.8660643],
[1,2.1815915],
[2,1.4881321],
[1,1.6342332],
[2,1.5071036],
[1,1.5950464],
[1,1.2154874],
[4,1.5704111],
[1,1.4999061],
[1,0.95844793],
[2,1.0333613],
[0,0.82135975],
[1,1.3967812],
[1,1.0393807],
[1,1.1608542],
[2,0.669296],
[2,0.7003008]]





In [13]:



truth=[]
pred=[]
for item in zip(trainX,prediction):
    #print(item[0][0][2],item[1][0])
    truthTemp=[]
    predTemp=[]
    truthTemp.append(item[0][0][2])
    truthTemp.append(item[0][0][3])
    truthTemp.append(item[1][0])
    
    predTemp.append(item[0][0][2])
    predTemp.append(item[0][0][3])
    predTemp.append(item[1][1])
    truth.append(truthTemp)
    pred.append(predTemp)
    
truth
print("pred")
truth




pred


[[0.0, 0.0, 0],
 [0.0, 0.0, 0],
 [0.0, 0.0, 0],
 [0.0, 0.0, 0],
 [37.3974, -121.93575, 1],
 [37.398309999999995, -121.93213, 3],
 [37.3945, -121.9212, 0],
 [37.39754, -121.9131, 4],
 [37.3976, -121.91281000000001, 0],
 [37.3976, -121.91281000000001, 0],
 [37.400999999999996, -121.9085, 3],
 [37.404, -121.9094, 4],
 [37.4252, -121.9164, 1],
 [37.4361, -121.92, 2],
 [37.448, -121.9217, 1],
 [37.464, -121.927, 2],
 [37.4811, -121.9352, 1],
 [37.4922, -121.9483, 2],
 [37.5052, -121.9642, 1],
 [37.508, -121.963, 0],
 [37.508, -121.963, 0],
 [37.508, -121.963, 0],
 [37.5178, -121.9548, 2],
 [37.52, -121.958, 0],
 [37.52, -121.958, 0],
 [37.52, -121.958, 0],
 [37.5359, -121.965, 2],
 [37.5369, -121.96600000000001, 1],
 [37.5324, -121.9687, 3],
 [37.5324, -121.9687, 0],
 [37.5324, -121.9687, 0],
 [37.5324, -121.9687, 0],
 [37.5324, -121.9687, 0],
 [37.5058, -121.9654, 2],
 [37.4959, -121.95200000000001, 2],
 [37.4845, -121.9377, 1],
 [37.4714, -121.9303, 1],
 [37.4539, -121.92299999999999, 2],

In [14]:
# visualizing ground truth first

tskip=[]
tnew=[]
tslworn=[]
tworn=[]
theavy=[]

for elem in truth:
    if(elem[0]==0 or elem[1]==0):
        continue
    
    if(elem[2]==0):
        tempL=[]
        tempL.append(elem[0])
        tempL.append(elem[1])
        tskip.append(tempL)
    elif(elem[2]==1):
        tempL=[]
        tempL.append(elem[0])
        tempL.append(elem[1])
        tnew.append(tempL)
    elif(elem[2]==2):
        tempL=[]
        tempL.append(elem[0])
        tempL.append(elem[1])
        tslworn.append(tempL)
    elif(elem[2]==3):
        tempL=[]
        tempL.append(elem[0])
        tempL.append(elem[1])
        tworn.append(tempL)
    elif(elem[2]==4):
        tempL=[]
        tempL.append(elem[0])
        tempL.append(elem[1])
        theavy.append(tempL)
        
        
    

In [15]:
print(tskip)
lat0=[]
long0=[]

for item in tskip:
    print(item[0],item[1])
    lat0.append(item[0])
    
    long0.append(item[0])
    
    
    
lat0=[]
long0=[]

for item in tskip:
    lat0.append(item[0])
    long0.append(item[1])

print(lat0)
print(long0)

[[37.3945, -121.9212], [37.3976, -121.91281000000001], [37.3976, -121.91281000000001], [37.508, -121.963], [37.508, -121.963], [37.508, -121.963], [37.52, -121.958], [37.52, -121.958], [37.52, -121.958], [37.5324, -121.9687], [37.5324, -121.9687], [37.5324, -121.9687], [37.5324, -121.9687], [37.40043, -121.995]]
37.3945 -121.9212
37.3976 -121.91281000000001
37.3976 -121.91281000000001
37.508 -121.963
37.508 -121.963
37.508 -121.963
37.52 -121.958
37.52 -121.958
37.52 -121.958
37.5324 -121.9687
37.5324 -121.9687
37.5324 -121.9687
37.5324 -121.9687
37.40043 -121.995
[37.3945, 37.3976, 37.3976, 37.508, 37.508, 37.508, 37.52, 37.52, 37.52, 37.5324, 37.5324, 37.5324, 37.5324, 37.40043]
[-121.9212, -121.91281000000001, -121.91281000000001, -121.963, -121.963, -121.963, -121.958, -121.958, -121.958, -121.9687, -121.9687, -121.9687, -121.9687, -121.995]


# Visualizing Test data ground truth on google maps.

#### Please replace plot.api_key with your own key

In [16]:
# ADDING ONE AT A TIME
# ['#440154', '#404387', '#29788E', '#22A784', '#79D151', '#FDE724']

from bokeh.io import output_file, output_notebook, show
from bokeh.models import (
  GMapPlot, GMapOptions, ColumnDataSource, Circle, LogColorMapper, BasicTicker, ColorBar,
    DataRange1d, PanTool, WheelZoomTool, BoxSelectTool
)
from bokeh.models.mappers import ColorMapper, LinearColorMapper
from bokeh.palettes import Viridis5



from bokeh.io import output_file, show
from bokeh.models import (
  GMapPlot, GMapOptions, ColumnDataSource, Circle, Range1d, PanTool, WheelZoomTool, BoxSelectTool
)

map_options = GMapOptions(lat=37.5324, lng=-121.9687, map_type="roadmap", zoom=11)

plot = GMapPlot(x_range=Range1d(), y_range=Range1d(), map_options=map_options)
plot.title.text = "San Francisco Bay Area"

# For GMaps to function, Google requires you obtain and enable an API key:
#
#     https://developers.google.com/maps/documentation/javascript/get-api-key
#
# Replace the value below with your personal API key:

plot.api_key = "key"

lat0=[]
long0=[]

for item in tskip:
    lat0.append(item[0])
    long0.append(item[1])


source = ColumnDataSource(
    data=dict(
        lat=lat0,
        lon=long0,
    )
)

circle = Circle(x="lon", y="lat", size=6, fill_color="black", fill_alpha=1.8, line_color=None)
plot.add_glyph(source, circle)
#output_notebook()
#plot.add_tools(PanTool(), WheelZoomTool(), BoxSelectTool())
#output_file("gmap_plot.html")
#show(plot)


# *****************************************************************************************************

# # FOR NEW ROAD

lat1=[]
long1=[]

for item in tnew:
    lat1.append(item[0])
    long1.append(item[1])


source1 = ColumnDataSource(
    data=dict(
        lat1=lat1,
        lon1=long1,
    )
)

circle1 = Circle(x="lon1", y="lat1", size=16, fill_color="green", fill_alpha=1.8, line_color=None)
plot.add_glyph(source1, circle1)
#output_notebook()
#plot.add_tools(PanTool(), WheelZoomTool(), BoxSelectTool())
#output_file("gmap_plot.html")
#show(plot)




# *****************************************************************************************************

# # FOR SLIGHTLY WORN ROAD

lat2=[]
long2=[]

for item in tslworn:
    lat2.append(item[0])
    long2.append(item[1])


source2 = ColumnDataSource(
    data=dict(
        lat2=lat2,
        lon2=long2,
    )
)

circle2 = Circle(x="lon2", y="lat2", size=13, fill_color="yellow", fill_alpha=1.8, line_color=None)
plot.add_glyph(source2, circle2)
#output_notebook()
#plot.add_tools(PanTool(), WheelZoomTool(), BoxSelectTool())
#output_file("gmap_plot.html")
#show(plot)






# *****************************************************************************************************

# # FOR  WORN ROAD

lat3=[]
long3=[]

for item in tworn:
    lat3.append(item[0])
    long3.append(item[1])


source3 = ColumnDataSource(
    data=dict(
        lat3=lat3,
        lon3=long3,
    )
)

circle3 = Circle(x="lon3", y="lat3", size=10, fill_color="purple", fill_alpha=1.8, line_color=None)
plot.add_glyph(source3, circle3)
#output_notebook()
#plot.add_tools(PanTool(), WheelZoomTool(), BoxSelectTool())
#output_file("gmap_plot.html")
#show(plot)









# *****************************************************************************************************

# # FOR  HEAVY ROAD

lat4=[]
long4=[]

for item in theavy:
    lat4.append(item[0])
    long4.append(item[1])


source4 = ColumnDataSource(
    data=dict(
        lat4=lat4,
        lon4=long4,
    )
)

circle4 = Circle(x="lon4", y="lat4", size=8, fill_color="red", fill_alpha=1.8, line_color=None)
plot.add_glyph(source4, circle4)
output_notebook()
plot.add_tools(PanTool(), WheelZoomTool(), BoxSelectTool())
#output_file("gmap_plot.html")
show(plot)



Loading BokehJS ...

In [17]:
pred

[[0.0, 0.0, 2.2048342],
 [0.0, 0.0, 1.1126734],
 [0.0, 0.0, 1.3997675],
 [0.0, 0.0, 0.5157764],
 [37.3974, -121.93575, 0.6071486],
 [37.398309999999995, -121.93213, 2.0375755],
 [37.3945, -121.9212, 0.99859166],
 [37.39754, -121.9131, 1.7473166],
 [37.3976, -121.91281000000001, 1.1756531],
 [37.3976, -121.91281000000001, 1.6052155],
 [37.400999999999996, -121.9085, 1.8819863],
 [37.404, -121.9094, 1.265181],
 [37.4252, -121.9164, 1.3298031],
 [37.4361, -121.92, 1.2512382],
 [37.448, -121.9217, 1.0782294],
 [37.464, -121.927, 1.5557319],
 [37.4811, -121.9352, 1.3530238],
 [37.4922, -121.9483, 1.0890144],
 [37.5052, -121.9642, 1.8473151],
 [37.508, -121.963, 0.8526046],
 [37.508, -121.963, 1.0932784],
 [37.508, -121.963, 1.2102916],
 [37.5178, -121.9548, 1.2795185],
 [37.52, -121.958, 0.67705584],
 [37.52, -121.958, 0.6167114],
 [37.52, -121.958, 0.6345362],
 [37.5359, -121.965, 0.8060204],
 [37.5369, -121.96600000000001, 1.1223748],
 [37.5324, -121.9687, 1.4328588],
 [37.5324, -121.9687

In [18]:
from bokeh.io import output_file, output_notebook, show
from bokeh.models import (
  GMapPlot, GMapOptions, ColumnDataSource, Circle, LogColorMapper, BasicTicker, ColorBar,
    DataRange1d, PanTool, WheelZoomTool, BoxSelectTool
)
from bokeh.models.mappers import ColorMapper, LinearColorMapper
from bokeh.palettes import Viridis5



from bokeh.io import output_file, show
from bokeh.models import (
  GMapPlot, GMapOptions, ColumnDataSource, Circle, Range1d, PanTool, WheelZoomTool, BoxSelectTool
)

# visualizing predictions

pskip=[]
pnew=[]
pslworn=[]
pworn=[]
pheavy=[]

for elem in pred:
    if(elem[0]==0 or elem[1]==0):
        continue
    
    if(elem[2]>=0 and elem[2]<=0.5):
        tempL=[]
        tempL.append(elem[0])
        tempL.append(elem[1])
        pskip.append(tempL)
    elif(elem[2]>0.5 and elem[2]<=1.5):
        tempL=[]
        tempL.append(elem[0])
        tempL.append(elem[1])
        pnew.append(tempL)
    elif(elem[2]>1.5 and elem[2]<=2.5):
        tempL=[]
        tempL.append(elem[0])
        tempL.append(elem[1])
        pslworn.append(tempL)
    elif(elem[2]>2.5 and elem[2]<=3.5):
        tempL=[]
        tempL.append(elem[0])
        tempL.append(elem[1])
        pworn.append(tempL)
    elif(elem[2]>3.5):
        tempL=[]
        tempL.append(elem[0])
        tempL.append(elem[1])
        pheavy.append(tempL)
        
        

        


# Visualizing Test data predictions on google maps.

#### Please replace plot.api_key with your own key

In [19]:
# predictions plot_pred
# ADDING ONE AT A TIME
# ['#440154', '#404387', '#29788E', '#22A784', '#79D151', '#FDE724']



        
        
        
        
        
        
        
        
        
        


map_options = GMapOptions(lat=37.5324, lng=-121.9687, map_type="roadmap", zoom=11)

plot_pred = GMapPlot(x_range=Range1d(), y_range=Range1d(), map_options=map_options)
plot_pred.title.text = "San Francisco Bay Area"

# For GMaps to function, Google requires you obtain and enable an API key:
#
#     https://developers.google.com/maps/documentation/javascript/get-api-key
#
# Replace the value below with your personal API key:

plot_pred.api_key = "key"

lat0=[]
long0=[]

for item in pskip:
    lat0.append(item[0])
    long0.append(item[1])


source = ColumnDataSource(
    data=dict(
        lat=lat0,
        lon=long0,
    )
)

circle = Circle(x="lon", y="lat", size=6, fill_color="black", fill_alpha=1.8, line_color=None)
plot_pred.add_glyph(source, circle)
#output_notebook()
#plot_pred.add_tools(PanTool(), WheelZoomTool(), BoxSelectTool())
#output_file("gmap_plot_pred.html")
#show(plot_pred)


# *****************************************************************************************************

# # FOR NEW ROAD

lat1=[]
long1=[]

for item in pnew:
    lat1.append(item[0])
    long1.append(item[1])


source1 = ColumnDataSource(
    data=dict(
        lat1=lat1,
        lon1=long1,
    )
)

circle1 = Circle(x="lon1", y="lat1", size=16, fill_color="green", fill_alpha=1.8, line_color=None)
plot_pred.add_glyph(source1, circle1)
#output_notebook()
#plot_pred.add_tools(PanTool(), WheelZoomTool(), BoxSelectTool())
#output_file("gmap_plot_pred.html")
#show(plot_pred)




# *****************************************************************************************************

# # FOR SLIGHTLY WORN ROAD

lat2=[]
long2=[]

for item in pslworn:
    lat2.append(item[0])
    long2.append(item[1])


source2 = ColumnDataSource(
    data=dict(
        lat2=lat2,
        lon2=long2,
    )
)

circle2 = Circle(x="lon2", y="lat2", size=13, fill_color="yellow", fill_alpha=1.8, line_color=None)
plot_pred.add_glyph(source2, circle2)
#output_notebook()
#plot_pred.add_tools(PanTool(), WheelZoomTool(), BoxSelectTool())
#output_file("gmap_plot_pred.html")
#show(plot_pred)






# *****************************************************************************************************

# # FOR  WORN ROAD

lat3=[]
long3=[]

for item in pworn:
    lat3.append(item[0])
    long3.append(item[1])


source3 = ColumnDataSource(
    data=dict(
        lat3=lat3,
        lon3=long3,
    )
)

circle3 = Circle(x="lon3", y="lat3", size=10, fill_color="purple", fill_alpha=1.8, line_color=None)
plot_pred.add_glyph(source3, circle3)
#output_notebook()
#plot_pred.add_tools(PanTool(), WheelZoomTool(), BoxSelectTool())
#output_file("gmap_plot_pred.html")
#show(plot_pred)









# *****************************************************************************************************

# # FOR  HEAVY ROAD

lat4=[]
long4=[]

for item in pheavy:
    lat4.append(item[0])
    long4.append(item[1])


source4 = ColumnDataSource(
    data=dict(
        lat4=lat4,
        lon4=long4,
    )
)

circle4 = Circle(x="lon4", y="lat4", size=8, fill_color="red", fill_alpha=1.8, line_color=None)
plot_pred.add_glyph(source4, circle4)
output_notebook()
plot_pred.add_tools(PanTool(), WheelZoomTool(), BoxSelectTool())
#output_file("gmap_plot_pred.html")
show(plot_pred)


        



Loading BokehJS ...